In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))



DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
def po_linear_model_square_expo(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * (sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])**2   

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]                
        g_feat_list = []
        y_list = []
        
        for p in p_list:            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            nx.set_node_attributes(g, 0, "z")
            if len(tr_clusters) > 0:
                tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])                  
                nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            po_linear_model_square_expo(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    # torch.save(GATE_hat_list, "Result/2step_CR_cluster.pkl")
    torch.save(GATE_hat_list, "Result/Square_incre_2step_CR_cluster.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1246
Epoch 0, Treat Prop: 0.02, Loss: 2.3959
Epoch 50, Treat Prop: 0.00, Loss: 0.1825
Epoch 50, Treat Prop: 0.02, Loss: 0.1837
Epoch 100, Treat Prop: 0.00, Loss: 0.0238
Epoch 100, Treat Prop: 0.02, Loss: 0.0365
Epoch 150, Treat Prop: 0.00, Loss: 0.0133
Epoch 150, Treat Prop: 0.02, Loss: 0.0210
Epoch 200, Treat Prop: 0.00, Loss: 0.0122
Epoch 200, Treat Prop: 0.02, Loss: 0.0169
Epoch 250, Treat Prop: 0.00, Loss: 0.0118
Epoch 250, Treat Prop: 0.02, Loss: 0.0145
Epoch 300, Treat Prop: 0.00, Loss: 0.0115
Epoch 300, Treat Prop: 0.02, Loss: 0.0132
Epoch 350, Treat Prop: 0.00, Loss: 0.0105
Epoch 350, Treat Prop: 0.02, Loss: 0.0120
0.012006078846752644
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1190
Epoch 0, Treat Prop: 0.05, Loss: 2.4155
Epoch 50, Treat Prop: 0.00, Loss: 0.1678
Epoch 50, Treat Prop: 0.05, Loss: 0.2040
Epoch 100, Treat Prop: 0.00, Loss: 0.0242
Epoch 100, Treat Prop: 0.05, Loss: 0.0557
Epoch 150, Treat Prop: 0.00, Loss: 0.0140
Epoch 150, Treat 

  0%|          | 1/1000 [00:22<6:18:48, 22.75s/it]

...

0.045414961874485016
Seed: 234
Epoch 0, Treat Prop: 0.00, Loss: 8.1140
Epoch 0, Treat Prop: 0.02, Loss: 2.3824
Epoch 50, Treat Prop: 0.00, Loss: 0.1762
Epoch 50, Treat Prop: 0.02, Loss: 0.1821
Epoch 100, Treat Prop: 0.00, Loss: 0.0235
Epoch 100, Treat Prop: 0.02, Loss: 0.0383
Epoch 150, Treat Prop: 0.00, Loss: 0.0134
Epoch 150, Treat Prop: 0.02, Loss: 0.0220
Epoch 200, Treat Prop: 0.00, Loss: 0.0121
Epoch 200, Treat Prop: 0.02, Loss: 0.0177
Epoch 250, Treat Prop: 0.00, Loss: 0.0117
Epoch 250, Treat Prop: 0.02, Loss: 0.0153
Epoch 300, Treat Prop: 0.00, Loss: 0.0117
Epoch 300, Treat Prop: 0.02, Loss: 0.0136
Epoch 350, Treat Prop: 0.00, Loss: 0.0107
Epoch 350, Treat Prop: 0.02, Loss: 0.0124
0.02361195534467697
Seed: 234
Epoch 0, Treat Prop: 0.00, Loss: 8.1108
Epoch 0, Treat Prop: 0.05, Loss: 2.7989
Epoch 50, Treat Prop: 0.00, Loss: 0.1630
Epoch 50, Treat Prop: 0.05, Loss: 0.4581
Epoch 100, Treat Prop: 0.00, Loss: 0.0283
Epoch 100, Treat Prop: 0.05, Loss: 0.1916
Epoch 150, Treat Prop: 0.00

 24%|██▎       | 235/1000 [1:29:44<5:00:19, 23.56s/it]

0.05039582774043083
Seed: 235
Epoch 0, Treat Prop: 0.00, Loss: 8.1062
Epoch 0, Treat Prop: 0.02, Loss: 2.3869
Epoch 50, Treat Prop: 0.00, Loss: 0.1605
Epoch 50, Treat Prop: 0.02, Loss: 0.1808
Epoch 100, Treat Prop: 0.00, Loss: 0.0210
Epoch 100, Treat Prop: 0.02, Loss: 0.0368
Epoch 150, Treat Prop: 0.00, Loss: 0.0130
Epoch 150, Treat Prop: 0.02, Loss: 0.0222
Epoch 200, Treat Prop: 0.00, Loss: 0.0120
Epoch 200, Treat Prop: 0.02, Loss: 0.0175
Epoch 250, Treat Prop: 0.00, Loss: 0.0116
Epoch 250, Treat Prop: 0.02, Loss: 0.0149
Epoch 300, Treat Prop: 0.00, Loss: 0.0114
Epoch 300, Treat Prop: 0.02, Loss: 0.0133
Epoch 350, Treat Prop: 0.00, Loss: 0.0109
Epoch 350, Treat Prop: 0.02, Loss: 0.0126
0.011866691522300243
Seed: 235
Epoch 0, Treat Prop: 0.00, Loss: 8.1165
Epoch 0, Treat Prop: 0.05, Loss: 2.4002
Epoch 50, Treat Prop: 0.00, Loss: 0.1589
Epoch 50, Treat Prop: 0.05, Loss: 0.2405
Epoch 100, Treat Prop: 0.00, Loss: 0.0259
Epoch 100, Treat Prop: 0.05, Loss: 0.0752
Epoch 150, Treat Prop: 0.00

 24%|██▎       | 236/1000 [1:30:07<4:59:25, 23.51s/it]

0.03745552524924278
Seed: 236
Epoch 0, Treat Prop: 0.00, Loss: 8.1161
Epoch 0, Treat Prop: 0.02, Loss: 2.3772
Epoch 50, Treat Prop: 0.00, Loss: 0.1699
Epoch 50, Treat Prop: 0.02, Loss: 0.1795
